# FABRIC Example: Map of Resources


## Configure the Environment

This process may take a moment, as the folium mapping package must be installed to the kernel.

In [13]:
import os
os.system('conda install folium')
from fabrictestbed.slice_manager import SliceManager, Status
import json

credmgr_host = os.environ['FABRIC_CREDMGR_HOST']
orchestrator_host = os.environ['FABRIC_ORCHESTRATOR_HOST']
print(f"CM Host: {credmgr_host} Orchestrator Host: {orchestrator_host}")

#Configure SSH Key
ssh_key = None
with open ("/home/fabric/.ssh/id_rsa.pub", "r") as myfile:
    ssh_key=myfile.read().strip()

CM Host: beta-2.fabric-testbed.net Orchestrator Host: beta-7.fabric-testbed.net


### Save the Initial Refresh Token

In [14]:
#Retrieve or set the refresh token (exprires 24 hours after login)
not_found=False
fabric_refresh_token=None
%store -r fabric_refresh_token

if fabric_refresh_token is None:
    fabric_refresh_token=os.environ['CILOGON_REFRESH_TOKEN']
    %store fabric_refresh_token
print("Fabric Refresh Token {}".format(fabric_refresh_token))
print("CILOGON_REFRESH_TOKEN environment variable: {}".format(os.environ['CILOGON_REFRESH_TOKEN']))

Fabric Refresh Token NB2HI4DTHIXS6Y3JNRXWO33OFZXXEZZPN5QXK5DIGIXTCMLDGJRTCMDFGUZGCMTEGBQTIYJTGBQWKZBYGVSDQNBXMZRDKYR7OR4XAZJ5OJSWM4TFONUFI33LMVXCM5DTHUYTMMRYGI4DSOBTG44DKMBGOZSXE43JN5XD25RSFYYCM3DJMZSXI2LNMU6TQNRUGAYDAMBQ
CILOGON_REFRESH_TOKEN environment variable: NB2HI4DTHIXS6Y3JNRXWO33OFZXXEZZPN5QXK5DIGIXTCMJUHAYWGMRQGJRGGYJSGAYDEYTFGEYGKYLBGQ2DQNRTGE3TAMB7OR4XAZJ5OJSWM4TFONUFI33LMVXCM5DTHUYTMMRYGI3DKMZYGIYDQMRGOZSXE43JN5XD25RSFYYCM3DJMZSXI2LNMU6TQNRUGAYDAMBQ


## Create and Initialize Slice Manager Object
Users can request tokens with different Project and Scopes by altering `project_name` and `scope` parameters in the refresh call below.

In [26]:
slice_manager = SliceManager(oc_host = orchestrator_host, cm_host = credmgr_host, project_name = 'all', scope = 'all')
slice_manager.initialize()

### Refresh the ID Token

ID Tokens expire one hour after refresh. 

In [27]:
try:
    id_token, refresh_token = slice_manager.refresh_tokens()
except Exception as e:
    print("Exception occurred while getting tokens:{}".format(e))

fabric_refresh_token=slice_manager.get_refresh_token()
print()
print("New Refresh Token: {}".format(fabric_refresh_token))
print()
print("Stored new Refresh Token")
%store fabric_refresh_token


New Refresh Token: NB2HI4DTHIXS6Y3JNRXWO33OFZXXEZZPN5QXK5DIGIXTCZBVGI4TGMZXHBRWINTDMFSTCYRTGY4DONLEGA4DCZTBMM3GENR7OR4XAZJ5OJSWM4TFONUFI33LMVXCM5DTHUYTMMRYGI4DSOJVHAZDKNRGOZSXE43JN5XD25RSFYYCM3DJMZSXI2LNMU6TQNRUGAYDAMBQ

Stored new Refresh Token
Stored 'fabric_refresh_token' (str)


### Query Slices

In [32]:
status, slices = slice_manager.slices()

print("Response Status {}".format(status))
if status == Status.OK:
    print("Slices {}".format(slices))
else:
    print(f"Failure: {slices}")

Response Status Status.OK
Slices [{
    "graph_id": "2b6dba6b-fdd2-4658-8d4b-e46a874a8f7b",
    "lease_end": "2021-08-07 22:43:44",
    "slice_id": "d1664743-1748-4f5a-a565-1083e0a6855c",
    "slice_name": "Slice-l2bridge-sriov",
    "slice_state": "StableOK"
}]


### Get Slice Information, Query Slivers From Slices

In [33]:
slice_ids = [slice.slice_id for slice in slices]
slice_names = [slice.slice_name for slice in slices]
slice_lease_ends = [slice.lease_end for slice in slices]
clean_slice_lease_ends = []

months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
for sle in slice_lease_ends:
    year = sle[:4]
    month = months[int(sle[5:7])]
    day = sle[8:10]
    if day[0] == '0':
        day = day[1]
    hour = sle[11:13]
    ampm = "AM" if int(hour) <= 12 else "PM"
    if hour[0] == '0':
        hour = hour[1]
    if int(hour) > 12:
        hour = str(int(hour) - 12)
    minute = sle[14:16]
    second = sle[17:19]
    date = month + ' ' + day + ', ' + year + f' [{hour}:{minute}:{second} {ampm}]'
    clean_slice_lease_ends.append(date)
    

for slice_id in slice_ids:
    status, slivers = slice_manager.slivers(slice_id = slice_id)
    
    print(f'\nSlice Name: {slice_names[slice_ids.index(slice_id)]}\n')
    print("Response Status {}".format(status))
    if status == Status.OK:
        print("Slivers {}".format(slivers))
    else:
        print(f"Failure: {slivers}")


Slice Name: Slice-l2bridge-sriov

Response Status Status.OK
Slivers [{
    "graph_node_id": "1e945cf3-151c-41c6-90a2-9dc6c574eebe",
    "join_state": "NoJoin",
    "lease_end": "2021-08-07 22:43:44",
    "name": "bridge1",
    "pending_state": "None_",
    "reservation_id": "4b8f1bd5-a1cc-4fb3-934b-ea9a685e3bed",
    "reservation_state": "Active",
    "resource_type": "L2Bridge",
    "site": "LBNL",
    "slice_id": "d1664743-1748-4f5a-a565-1083e0a6855c"
}, {
    "allocated_capacities": "{\"core\": 2, \"disk\": 10, \"ram\": 8}",
    "allocated_labels": "{\"instance\": \"instance-000000e5\", \"instance_parent\": \"lbnl-w2.fabric-testbed.net\"}",
    "capacities": "{\"core\": 2, \"disk\": 10, \"ram\": 6}",
    "capacity_hints": "{\"instance_type\": \"fabric.c2.m8.d10\"}",
    "graph_node_id": "0e38eb30-7b59-4eee-9563-0361f6e1b023",
    "join_state": "NoJoin",
    "lease_end": "2021-08-07 22:43:44",
    "management_ip": "198.129.61.37",
    "name": "node2",
    "pending_state": "None_",
 

### Create and Save Map

- Click radio buttons to generate map for different slices
- Click markers to view details about individual resources
- Map saves as interactable HTML file

In [44]:
site_dict = {'UKY': (38.03154264461649, -84.50343326019218), 'RENC': (35.93996154497003, -79.01809264307425), 'LBNL': (38.94977311613871, -122.61289106281062)}

import folium
from folium.plugins import MarkerCluster
from folium.features import DivIcon
import branca
from branca.element import Template, MacroElement

base = folium.Map(location = (38.12480976137421, -95.7129), zoom_start = 4.2, zoom_control = False, dragging = False)

for slice_id in slice_ids:
    status, slivers = slice_manager.slivers(slice_id = slice_id)

    labels = {}
    for sliver in slivers:
        status, sliver_status = slice_manager.sliver_status(slice_id = slice_id, sliver_id = sliver.reservation_id)
        if sliver_status.resource_type == "VM":
            name = sliver_status.name
            site = sliver_status.site
            capacities = sliver_status.capacities
            state = sliver_status.reservation_state
            info = (site, capacities, site_dict[site], state)
            labels[name] = info

    locations = []
    popups = []
    icons = []

    for label in labels:
        locations.append(site_dict[labels[label][0]])
        popups.append(folium.Popup(branca.element.IFrame(html = f'<strong style="font-family: Verdana">Resource: {label} [{labels[label][3]}]</strong><br>Site: {labels[label][0]}<br>Capacities: {labels[label][1]}<br>\nLease Ends: {clean_slice_lease_ends[slice_ids.index(slice_id)]}', width = 350, height = 90), max_width = 350))
        color = None
        if labels[label][3] == 'Active':
            color = 'green'
        elif labels[label][3] == 'Closing':
            color = 'orange'
        else:
            color = 'red'
        icons.append(folium.Icon(color = color, icon = 'server', prefix = 'fa'))
        
    cluster = MarkerCluster(name = slice_names[slice_ids.index(slice_id)], 
                            locations = locations, 
                            popups = popups, 
                            icons = icons,
                            show = False)
    folium.map.Marker((52, -110), 
                      icon = DivIcon(icon_size = (300, 60),
                                     icon_anchor = (0, 0),
                                     html = f'<div style="font-size: 20pt;font-family: Verdana;text-align: center;border: 1pt solid lightgray;background: ghostwhite">{slice_names[slice_ids.index(slice_id)]}</div>')).add_to(cluster)
    cluster.add_to(base)
    
    
    
                                                 
folium.LayerControl(collapsed = False).add_to(base)
folium.map.Marker((53.85, -77.7), 
                      icon = DivIcon(icon_size = (110, 60),
                                     icon_anchor = (0, 0),
                                     html = '<div style="font-size: 10pt;font-family: Verdana;text-align: center;border: 1pt solid lightgray;background: ghostwhite">Click to generate map for slice</div>')).add_to(base)

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px;font-family: Verdana; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Legend</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:green;opacity:0.7;'></span>Active</li>
    <li><span style='background:orange;opacity:0.7;'></span>Closing</li>
    <li><span style='background:red;opacity:0.7;'></span>Dead</li>

  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
    
</style>
{% endmacro %}
"""


macro = MacroElement()
macro._template = Template(template)

base.get_root().add_child(macro)

base.save('map.html')
base